In [11]:
englishLetterFreq = {'E': .1202, 'T': .091, 'A': .0812, 'O': .0768, 'I': .0731, 'N': .0695, 'S': .0628, 'H': .0592, 'R': .0602, 'D': .0432, 'L': .0398, 'C': .0271, 'U': .0288, 'M': .0261, 'W': .0209, 'F': .0230, 'G': .0203, 'Y': .0211, 'P': .0182, 'B': .0149, 'V': .0111, 'K': .0069, 'J': .0010, 'X': .0017, 'Q': .0011, 'Z': .0007}

In [12]:
caracteres = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
transformador_a_numero = {c: caracteres.index(c) for c in caracteres}
transformador_a_letra = {caracteres.index(c):c for c in caracteres}

## Funcion para encontrar el largo de la llave

Esta funcion lo que hace es ir comparando las distribuciones de cada largo de llave para que se asemejen a las frecuencias entregadas, no es necesario decriptar con cada llave, sino que solamente ir contando cada letra encriptada, esto es debido a que OTP es 1-1, por lo que la frecuencia se deberia mantener similar (la distribucion en si, sin contar que letra corresponde a que frequencia).

El razonamiento es el siguiente: para cada largo contamos sus frecuencias, una vez que ya la tenemos la ordenamos de mayor a menor, al igual que la frecuencia entregada. Una vez las dos frecuencias ordenadas, las comparamos y el largo que tenga la menor diferencia, es el largo de la llave.

In [13]:
def most_likely_key_length(ciphertext,
    frequencies,
    distance):
    max_length = len(ciphertext)//50
    real_key_length = 1
    
    alphabet = sorted(frequencies.keys())
    freq_sort = sorted(frequencies.values())

    # ordenamos las frecuencia por su valor
    max_diff = 10000000000000000000000000000000000000000000000000000000000000000000000000
    
    # iteramos sobre todos los largos de llaves posibles
    for length in range(1,max_length+1):
        suma = 0
        # iteramos sobre el largo de la llave
        for pos in range(length):
            compare_dict = {k:0 for k in frequencies.keys()}
            pos_len = len(ciphertext)//length

            # separamos el ciphertext por las letras que corresponden a esa parte de la llave
            to_decrypt = [ciphertext[i*length + pos] for i in range(pos_len)]           

            for letter in to_decrypt:
                compare_dict[letter] += 1
                # sumamos las frecuencias de cada letra

            sorted_count = sorted(compare_dict.values())
            # ordenamos la frecuencia obtenida por sus valores (no por las letras) al igual que como ordenamos antes
            # la frecuencia de input
            for i in range(len(sorted_count)):
                # para cada valor sumamos la diferencia promediada segun el largo de las ocurrencias de esa parte de la llave
                if i < len(sorted_count):
                    suma += abs((freq_sort[i] - sorted_count[i]/pos_len))
                else:
                    suma += freq_sort[i]
        # promediamos por el largo de la llave
        suma /= length
        if suma < max_diff:
            real_key_length = length
            max_diff = suma
    return real_key_length

## Encrypt y Decrypt OTP

Estas funciones las cree para probar el codigo, las hice con la materia de clases

In [14]:
def encrypt_otp(texto,key):
    cypher_text = ""
    cypher_number = []
    new_key = ""
    while len(new_key) < len(texto):
        new_key += key
    
    key = new_key[0:len(texto)]



    for t,k in zip(texto,key):
        cypher_number.append((transformador_a_numero[t] + transformador_a_numero[k])%len(englishLetterFreq))
        cypher_text += transformador_a_letra[cypher_number[-1]]

    print(cypher_text)

In [15]:
def decrypt_otp(cipher,key):
    key = "".join(key)
    text = ""
    number = []
    new_key = ""
    while len(new_key) < len(cipher):
        new_key += key
    
    key = new_key[0:len(cipher)]
    
    for t,k in zip(cipher,key):
        number.append((transformador_a_numero[t] - transformador_a_numero[k])%len(englishLetterFreq))
        text += transformador_a_letra[number[-1]]

    return text


## Funcion para sacar el caracter mas probable

Para esta funcion me inspire en la funcion creada en clases para decriptar otp con un largo en especifico, lo que hace es que para cada letra del alphabeto decripta una parte del texto cifrado, y va probando cual tiene la mejor frecuencia

In [16]:
def most_likely_char(pos, ciphertext,key_len,frequencies,distance):
    to_decrypt = [ciphertext[i*key_len + pos] for i in range(len(ciphertext)//key_len)]
    # separamos el ciphertext por las letras que corresponden a esa parte de la llave
    alphabet = sorted(frequencies.keys())
    best_char = alphabet[0]
    best_distance = len(alphabet)

    # iteramos por cada letra del alfabeto
    for candidate in alphabet:
        string = ''
        # iteramos por cada letra que corresponden a esa parte de la llave
        for c in to_decrypt:
            n_c = alphabet.index(c)
            n_candidate = alphabet.index(candidate)
            n = (n_c - n_candidate) % len(alphabet)
            # la decriptamos y añadimos al string de resultado
            string += alphabet[n]
        # comparamos la diferencia
        dif = distance(string,frequencies)
        if best_distance > dif:
            best_char = candidate
            best_distance = dif
    return best_char
            
            
    

In [17]:
def abs_distance ( string : str , frequencies) -> float :
    return sum([
    abs ( frequencies [c] - string . count ( c ) / len ( string ) )
    for c in frequencies
    ])


## BREAK OTP

Esta es la funcion a correr, lo primero que hace es sacar el largo de la llave con la funcion descrita mas arriba, una vez que lo tiene saca la llave con la funcion vista en clases

In [18]:
def break_rp(
    ciphertext,
    frequencies,
    distance,
) -> str:
    # sacamos el largo de la llave
    key_len = most_likely_key_length(
        ciphertext,
        frequencies,
        distance,
    )
    key = ''
    # sacamos la llave
    for i in range(key_len):
        key += most_likely_char(i, ciphertext, key_len, frequencies, distance)
    return key


In [19]:
cipher = "TIEFNJTPDLIYGEOXOGGCEBTMRJTLIOAYDOOCTIECNJRPLBNOOSUVITIYWFSEESNPUSOAEJTNONPCITEDTIETSMAYDPFRRFAEBSIEAJNPNHLLNESNOULLNEAYDXAWETAYDUHPNPREHFRYOOEDIYTSOGTSEJSWAODZFJRPLBNONPREHFRYISEWAODEOHEEHFRHIUHXAOYDMBLWESIDLBNOSUHPMBIYLBNOASELSMIPBFTHEFNWAUIEUEEDNBNONUHPSIEELBNOITLLNESCEBCSTPNPASLJNBNOLPNRIUUOETWEOFTSESOJAMGCEFNHIDHZBTECVBTZRZNPASLZNEOYITTSEEEQIOIYGQOTNUOQTIEARJMPMFRTDJAYTIEFNJTPDLIYGEOXHBSLTPTLLBRPAPFLPQRZXJMLTFLJKNTSEVKWIFSMEUWPEOTSEOOCTIAELBNEIDAYDUHPNPREHTELAODNONEDWJTSIOKXMJOQTIEYOSTSWFSECPADTPFQRBNNEGRZMXHTCIIEITSPPBRLTFDMYUHPEOGWITHNHBNYEMIESIACETAVMJNEESNLTJOYAMLLNEBZUODLRZWTTITSESEAUCLTCPFTRFLLNETSEDHLNOEWTVNYEMBZRFDMEOELTITSEFNRLJSSCIAYNFLYOXLTNLSEHFUVWJTSFSAYCFTSEQHJSJCLLHEZGSAAHZOQTIEFKWACIFSRRFAELZEYGMAYDDOYSJSESPFXOTTWYMOHLBNOTFRCAJNHIUHFPMAYDPRXOVNEAJNZUTTPRSATNPNWYGOFNENZRUHHETTZFUHPTFEDEYEWIOEEHFUALBNOASELSJNNLVDPTIEWALEOITTCIDTEHFPPNOIYETNZRUHJOSKXOPRDEYMZOSAYDEACTNOZRUHPLPWWAODLRFADASEEYQINAMLJTSAGESSPDCYCAOGPSPFWOXHTLMSQRFQFEOTWYDOXPPSPDPFNHBLVAODQLBTALBIYSTCZTMAYDJSEHFMZSUMZUOTLIOOFSDOFNURJIOTSEVKLNEIESQHJSJCLLHEZGSAAHZIDDJSEIOGFITHPDCYEHFHTGILLNEBZUODLRZFLUMTHHJCSTSAGESSPSUHPSDOETJSSMBIYLBNOFSOXHFLPNTBFRHHEOTTZNFHLVFNEHFFLUMTWIOEDEQACAUEDTIEEWPDTSUIYCUIGEMYOIGFPRFNERFGTOOSZFUHPHJGSLBNOSUOEHFNZRUHLNEWPSUAYDUHPLPWWAODDTPTSETOFTIAYDFADTUHPHJGSLBNOSBRPPSEOONIYAOTWYNOFNUATNPUDCPNEAJNTNHTSENAUOSIEYPFDCPTWAODDMPUYTBIYOVSWAODDCBPPWIIWEUHPLPWWAODDCPNEAJNQLBTEESLLNEEDPFCTAMLJADRZSTTSEDEYTSAWLPWWAODDWJTSUQLLNEAYDNOFNUATNPUDTFRCAJNWODAEEEAETIEDOVTSESNFPMAYDTWLLFSTSNODTMYXOVNEAJNZUTTSOVGSSPUEHXAWETIDLFSDMPUYTBIYOVSEHBNYOSTSAODXIEWLLFSYOSTSESNTRFLLNECZNTIDTTOQMPSELZHTLMYWAODDCBPPAODTTTGPOHRLPIYTNDLFDFSEHFMZUSNPMPUYTBIYSBSHEMLLSMOFGINPAHHLTTQFASEVIMOXEURPSTQXIUHPLBRRETTMOEYZFXAEESIYTIEFKUHPOWECAMLREPMZRQHZLPGJOGTSEVKHATSSAQEOBZANONBTNBTTOOOQFPRNETIYCMUOIOGEEDTZNJCDAODNLJMLTFCSAOGPIOPLRUINUMACGMANIBTTOOIYNPREHFRYAODHETTPROACEBSEHFTLLMEDTNOFNUATNJNEHFUVAODMRJTTSIIDLFSTSCEYNFVTSJNEHFGCANPTAOMZUOTLIOSDCPTWAODEHFLZNHEDTSIGESIDTIECIWECSFVPROWSIDHQLPWDFSOXWBLPSJNEOFNRLBNOTIEWASGPSULLKFBJSVRQADELRFATSMOFGINPAHHTNOOCTIECNJRPLBNOTIOFGISNOULLNESWODHYETSSATTSEMACGFSEVPLFMF"

In [20]:
print(break_rp (
    cipher,
    englishLetterFreq,
    abs_distance,
))

ABAL
